Assignment Id.AId.A

In [32]:
import numpy as np
import sys
import enum
class Id(enum.Enum):
    A = np.uint8(25)
    B = np.uint8(30)
    C = np.uint8(35)
    D = np.uint8(40)

print(sys.getrecursionlimit())
sys.setrecursionlimit(10**6)
print(sys.getrecursionlimit())

class unit:
    def __init__(self, x, y, id,  stepcost):
        self.x = x
        self.y = y
        self.id = id
        self.stepcost = stepcost
        self.hallway = False

    def copy(self):
        tmp = unit(self.x, self.y, self.id, self.stepcost)
        tmp.hallway = self.hallway
        return tmp
    

def GetData(dataset):
    data = []
    with open(dataset, 'r') as f:
        data = f.readlines()

    
    # strip the new line character
    #on x=-Id.A0..Id.A6,y=-Id.B6..17,z=-Id.C7..7

    grid=np.zeros(shape=(len(data),len(data[0])), dtype=np.uint8)
    list_of_units = []
    for y in range(len(data)):
        for x in range(len(data[y])):
            if data[y][x] == '#':
                grid[y][x] = 1
            elif data[y][x] == '.':
                grid[y][x] = 0
            elif data[y][x] == ',':
                grid[y][x] = 2
            elif data[y][x] == 'A':
                grid[y][x] = Id.A.value
            elif data[y][x] == 'B':
                grid[y][x] = Id.B.value
            elif data[y][x] == 'C':
                grid[y][x] = Id.C.value
            elif data[y][x] == 'D':
                grid[y][x] = Id.D.value
            else:
                grid[y][x] = 0
    return grid

gridexpected = GetData('./data/aoc23_expected.txt')
gridconfig = GetData('./data/aoc23_config.txt')
def ExpectedConfiguration():
    return gridexpected

def TestConfiguration():
    return gridconfig

def Scan():
    return np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,0],
                    [1,0,0,0,0,0,0,0,0,0,0,0,1,0],
                    [1,1,1,0,1,0,1,0,1,0,1,1,1,0],
                    [0,0,1,0,1,0,1,0,1,0,1,0,0,0],
                    [0,0,1,1,1,1,1,1,1,1,1,0,0,0]], dtype=np.uint8)


def GetUnits(grid):
    units = []
    for x in range(len(grid)):
        for y in range(len(grid[x])):
            #print(grid[x][y], x, y)
            if grid[x][y] == Id.A.value:
                print("A", x, y)
                units.append(unit(x, y,Id.A,1))
            elif grid[x][y] == Id.B.value:
                print("B", x, y)
                units.append(unit(x, y,Id.B,10))
            elif grid[x][y] == Id.C.value:
                print("C", x, y)
                units.append(unit(x, y,Id.C,100))
            elif grid[x][y]  == Id.D.value:
                print("D", x, y)
                units.append(unit(x, y,Id.D,1000))
    return units
# true if path is blocked
def CheckPathNotBlocked(grid, start, move, scanmap):
    
    #print(start[0],start[1], scanmap[start[0]][start[1]])
    
    if grid[start[0]][start[1]] >= Id.A.value or grid[start[0]][start[1]] == 1:
        #print("collided")
        return  False, 0
    if scanmap[start[0]][start[1]]== 1:
        #print("passed")
        return  False, 0
    if start[0] == move[0] and start[1] == move[1]:
        #print("reached")
        return True, 0
    scanmap[start[0],start[1]] = 1
    #print(scanmap)
    return BreathPath(grid, start, move, scanmap)

def BreathPath(grid, start, move, scanmap):
    check , value = CheckPathNotBlocked(grid, (start[0]+1, start[1]), move, scanmap)
    if check:
        return True, value + 1
    check , value= CheckPathNotBlocked(grid, (start[0]-1, start[1]), move, scanmap)
    if check:
        return True,value + 1
    check , value = CheckPathNotBlocked(grid, (start[0], start[1]+1), move, scanmap)
    if check:
        return True,value + 1
    check , value = CheckPathNotBlocked(grid, (start[0], start[1]-1), move, scanmap)
    if check:
        return True,value + 1
    return False, 0

        

def GenerateMoves(unit, grid):
    config = TestConfiguration()
    # check if is in hallway 
    moves = []
    if config[unit.x][unit.y] == 0:
        unit.hallway = True
        #print(f"in Halway {unit.x},{unit.y}")
        for x in range(len(config)-1,-1,-1):
            for y in range(len(config[x])):
                if config[x][y] == unit.id.value:
                    check,value = BreathPath(grid, [unit.x, unit.y], [x,y], Scan())
                    #print(f"{unit.id} {unit.x},{unit.y} {x},{y} {check} {value}")
                    if check:
                        return [[x,y, value * unit.stepcost]]
                
    # Move into the hallway
    else:
        if(unit.hallway == False):
            for y in range(len(config[1])):
                if grid[1][y] == 0 and config[1][y] == 0:
                    check,value = BreathPath(grid, [unit.x, unit.y], [1,y], Scan())
                    #print(f'endvalue {value}')
                    if check:
                        moves.append([1,y, value * unit.stepcost])
                    
    return moves


def MoveUntilGrid(startgrid, expected):
    griddic = {}
    griddic[0] = [[GetUnits(startgrid),startgrid,hash(str(startgrid))]]
    i = -1
    foundValue = 0
    foundValues = {}
    foundValues[hash(str(startgrid))] = 0
    while len(griddic) >0:
        i += 1
        if foundValue != 0 and i > foundValue:
            break
        if i in griddic:
            print(f'stepcost {i} found: {foundValue}')
            gridStates = griddic[i]
            del griddic[i]
            while len(gridStates) > 0:
                stepcost = i
                units,grid,tmphash = gridStates.pop(0)
                if tmphash in foundValues:
                    if foundValues[tmphash] > stepcost:
                        continue
                #print(units)
                for j in range(len(units)):
                    unit = units[j]
                    moves = GenerateMoves(unit,grid)
                    #print(moves)
                    if moves != None:
                        for move in moves:
                            newgrid = grid.copy()
                            newgrid [unit.x][unit.y] = 0
                            newgrid [move[0], move[1]] = unit.id.value
                            newUnits = [u.copy() for u in units]
                            # get unit on this position
                            
                            newUnits[j].x = move[0]
                            newUnits[j].y = move[1]
                            newstepcost = move[2] + stepcost
                            #print(newstepcost, move[2], stepcost)
                            gridhash=hash(str(newgrid))

                            #print(f'grid {gridhash}')
                            #print(newgrid)
                            #print(newstepcost)
                            #print(unit.stepcost)
                            if gridhash == expected:
                                if foundValue != 0 and newstepcost > foundValue:
                                        continue
                                foundValue=newstepcost

                            if newstepcost not in griddic: griddic[newstepcost] =  []
                            if gridhash not in foundValues:
                                #print(f'new grid')
                                if foundValue != 0 and newstepcost > foundValue:
                                        continue
                                foundValues[gridhash] = newstepcost
                                griddic[newstepcost].append([newUnits,newgrid,gridhash])
                                
                            else:
                                if foundValues[gridhash] > newstepcost :
                                    if foundValue != 0 and newstepcost > foundValue:
                                        continue
                                    #print(f'old grid')
                                    foundValues[gridhash] = newstepcost
                                    griddic[newstepcost].append([newUnits,newgrid,gridhash])
                               
    return foundValue
                            


1000000
1000000


In [33]:
data = GetData('./data/aoc23_test.txt')
config = hash(str(ExpectedConfiguration()))
print(MoveUntilGrid(data, config))


B 2 3
C 2 5
B 2 7
D 2 9
A 3 3
D 3 5
C 3 7
A 3 9
stepcost 0 found: 0
stepcost 20 found: 0
stepcost 23 found: 0
stepcost 24 found: 0
stepcost 25 found: 0
stepcost 26 found: 0
stepcost 27 found: 0
stepcost 28 found: 0
stepcost 29 found: 0
stepcost 30 found: 0
stepcost 33 found: 0
stepcost 34 found: 0
stepcost 35 found: 0
stepcost 36 found: 0
stepcost 37 found: 0
stepcost 38 found: 0
stepcost 39 found: 0
stepcost 40 found: 0
stepcost 43 found: 0
stepcost 44 found: 0
stepcost 45 found: 0
stepcost 46 found: 0
stepcost 47 found: 0
stepcost 48 found: 0
stepcost 49 found: 0
stepcost 50 found: 0
stepcost 53 found: 0
stepcost 55 found: 0
stepcost 56 found: 0
stepcost 57 found: 0
stepcost 59 found: 0
stepcost 60 found: 0
stepcost 63 found: 0
stepcost 64 found: 0
stepcost 65 found: 0
stepcost 66 found: 0
stepcost 67 found: 0
stepcost 68 found: 0
stepcost 69 found: 0
stepcost 70 found: 0
stepcost 73 found: 0
stepcost 74 found: 0
stepcost 75 found: 0
stepcost 76 found: 0
stepcost 77 found: 0
stepcost

KeyboardInterrupt: 

In [ ]:
data = GetData('./data/aoc23.txt')
config = hash(str(ExpectedConfiguration()))
print(MoveUntilGrid(data, config))

A 2 3
D 2 5
A 2 7
B 2 9
C 3 3
C 3 5
D 3 7
B 3 9
13495


In [ ]:
def Scan():
    return np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,0],
                    [1,0,0,0,0,0,0,0,0,0,0,0,1,0],
                    [1,1,1,0,1,0,1,0,1,0,1,1,1,0],
                    [0,0,1,0,1,0,1,0,1,0,1,0,0,0],
                    [0,0,1,0,1,0,1,0,1,0,1,0,0,0],
                    [0,0,1,0,1,0,1,0,1,0,1,0,0,0],
                    [0,0,1,1,1,1,1,1,1,1,1,0,0,0]], dtype=np.uint8)
data = GetData('./data/aoc23_p2_test.txt')
gridexpected = GetData('./data/aoc23_p2_expected.txt')
gridconfig = GetData('./data/aoc23_p2_config.txt')
config = hash(str(ExpectedConfiguration()))
print(MoveUntilGrid(data, config))

B 2 3
C 2 5
B 2 7
D 2 9
D 3 3
C 3 5
B 3 7
A 3 9
D 4 3
B 4 5
A 4 7
C 4 9
A 5 3
D 5 5
C 5 7
A 5 9
stepcost 0 found: 0
stepcost 20 found: 0
stepcost 30 found: 0
stepcost 40 found: 0
stepcost 50 found: 0
stepcost 60 found: 0
stepcost 70 found: 0
stepcost 74 found: 0
stepcost 76 found: 0
stepcost 77 found: 0
stepcost 78 found: 0
stepcost 79 found: 0
stepcost 80 found: 0
stepcost 84 found: 0
stepcost 86 found: 0
stepcost 88 found: 0
stepcost 89 found: 0
stepcost 90 found: 0
stepcost 94 found: 0
stepcost 96 found: 0
stepcost 97 found: 0
stepcost 98 found: 0
stepcost 99 found: 0
stepcost 100 found: 0
stepcost 102 found: 0
stepcost 104 found: 0
stepcost 106 found: 0
stepcost 107 found: 0
stepcost 108 found: 0
stepcost 109 found: 0
stepcost 110 found: 0
stepcost 112 found: 0
stepcost 114 found: 0
stepcost 116 found: 0
stepcost 117 found: 0
stepcost 118 found: 0
stepcost 119 found: 0
stepcost 120 found: 0
stepcost 122 found: 0
stepcost 124 found: 0
stepcost 126 found: 0
stepcost 127 found: 0
step

KeyboardInterrupt: 